In [3]:
import keras
from keras.models import Model,load_model
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score


In [4]:
model = load_model("cifar4_CNN_Epoch_100_Daug_True.h5")

In [5]:
NUM_CLASSES = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

# Normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 
x_test /= 255

In [6]:
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')


In [7]:
# adv_x_train = np.load("adversarial_x_train.npy")
# adv_y_train = np.load("adversarial_y_train.npy")
adv_x_test = np.load("adversarial_x_test.npy")
adv_y_test = np.load("adversarial_y_test.npy")

In [8]:
base_model = Model(inputs=model.input, outputs=model.layers[24].output)
x_train_flat = base_model.predict(x_train)

y_train_flat = np.argmax(y_train,axis=1)
x_test_flat =  base_model.predict(x_test)
y_test_flat =  np.argmax(y_test,axis=1)
x_test_adv_flat = base_model.predict(adv_x_test)
y_test_adv_flat = np.argmax(adv_y_test,axis=1)


In [9]:
clf_svm = svm.LinearSVC(random_state=0, tol=1e-3,verbose=1, C=0.1)
clf_svm.fit(x_train_flat, y_train_flat)

[LibLinear]

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.001, verbose=1)

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
clf_rf = RandomForestClassifier(n_estimators=100)

In [35]:
clf_rf.fit(x_train_flat, y_train_flat)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [38]:
print('RF Train accuracy:', accuracy_score(y_train_flat, clf_rf.predict(x_train_flat)))
print('RF Test accuracy:', accuracy_score(y_test_flat,clf_rf.predict(x_test_flat)))
print('RF Adversarial accuracy:', accuracy_score(y_test_adv_flat,clf_rf.predict(x_test_adv_flat)))

RF Train accuracy: 1.0
RF Test accuracy: 0.8494
RF Adversarial accuracy: 0.5083056478405316


In [39]:
print('CNN Train accuracy:', model.evaluate(x_train,y_train,verbose=0)[1])
print('CNN Test accuracy:', model.evaluate(x_test,y_test,verbose=0)[1])
print('CNN Adversarial accuracy:',  model.evaluate(adv_x_test,adv_y_test,verbose=0)[1])

CNN Train accuracy: 0.9325
CNN Test accuracy: 0.8828
CNN Adversarial accuracy: 0.0


In [40]:
print('SVM Train accuracy:', accuracy_score(y_train_flat, clf_svm.predict(x_train_flat)))
print('SVM Test accuracy:', accuracy_score(y_test_flat,clf_svm.predict(x_test_flat)))
print('SVM Adversarial accuracy:', accuracy_score(y_test_adv_flat,clf_svm.predict(x_test_adv_flat)))

SVM Train accuracy: 0.93302
SVM Test accuracy: 0.884
SVM Adversarial accuracy: 0.46572032618544246
